# Data Cleaning Strategy
For this project, we will use the CTN0027 Dataset from the clinical trial network.<br>
The dataset and all it's documentation are avaialble at the following website:<br>
https://datashare.nida.nih.gov/study/nida-ctn-0027<br>

There are a few challenges to highlight that should be considered in the cleaning approach:
- Large de-identified dataset must be manually labeled, very time consuming and prone to errors
- High dimension data - We will load, clean and transform 10 tables to create approx. 430 features 
per patient

## Data Cleaning Process
We will try to keep things simple and employ a process driven by reusable functions<br>
to speed up total cleaning time and reduce errors.<br>
<br> 
For each table we will follow the following steps:<br>
1. Load the data
2. Identify columns that require labels
3. Apply labels to columns
4. Drop columns that are not needed
5. Create imputation strategy for missing values
5. Apply transformations to values where required
3. Feature Engineering (if necessary)
4. Flatten Dataframes (encode week of treatment into columns, where applicable)
4. Merge with other tables

## List of Reusable Functions
| Name of Function | Description | 
| ---------------- | ----------- |
| clean_df | Clean the given DataFrame by dropping unnecessary columns, renaming columns, and reordering columns. |
| flatten_dataframe | This function creates features by combining the VISIT column with the clinical datapoint.  e.g. Transform `test_opiates` and `WK1` of treatment to `test_opiates_1`.  The VISIT column contains 25 weeks of data stored as 25 rows per patient.  This function encodes the week of treatment into features in columns, removing the VISIT column and 25,000 rows, thus flattening the dataframe |
| merge_dfs | Merge the given list of DataFrames into one DataFrame. |
| uds_features | Creates 3 new features which are metrics used to measure outcomes from opiate test data. |
| med_features | Creates 2 new features for medication dose to enrich dataset and improve accuracy in machine learning|

### Example of How Flattening Works
![flatten](../images/flatten.png)


## Tables to be cleaned
| File Name | Table Name | Variable |Description | Process Applied |
| :--- | :--- | :--- | :--- | :--- |
| T_FRRSA.csv | Research Session Attendance|RSA |Records attendence for each week of treatment | Clean, Flatten, Feature Extraction, Merge |
| T_FRDEM.csv | Demographics|DEM |Sex, Ethnicity, Race | Clean, Merge |
| T_FRUDSAB.csv | Urine Drug Screen| UDS  |Drug test for 8 different drug classes, taken weekly for 24 weeks | Clean, Flatten, Feature Extraction, Merge |
| T_FRDSM.csv | DSM-IV Diagnosis|DSM |Tracks 6 different conditions| Clean, Merge |
| T_FRMDH.csv | Medical and Psychiatric History|MDH |Tracks 24 different Conditions| Clean, Merge |
| T_FRPEX.csv | Physical Exam|PEX |Tracks 12 different physical observations| Clean, Merge |
| T_FRPBC_BL.csv | Pregnancy and Birth Control|PBC |Pregnancy test taken once per month on weeks 0, 4, 8, 12, 16, 20, 24| Clean, Merge |
| T_FRTFB.csv | Timeline Follow Back Survey|TFB |Surveys for self reported drug use, collected every 4 weeks, includes previous 30 days of use ot week 0, 4, 8, 12, 16, 20, 24| Clean, Aggregate, Flatten, Merge |
|T_FRDOS.csv | Dose Record |DOS |Records the dose of medication taken each week for 24 weeks| Clean, Aggregate, Feature Extraction, Flatten, Merge |
|SAE.csv | Serious Adverse Events |SAE |Records any serious adverse events that occur during the study| Clean, Merge |

### Import Required Libraries

In [1]:
import pandas as pd # data manipulation library
import numpy as np # numerical computing library
import matplotlib.pyplot as plt # data visualization library
import seaborn as sns # advanced data visualization library
import helper # custom fuctions I created to clean and plot data

import warnings
warnings.filterwarnings('ignore')

### Load the Data
We will load 10 files from the de-identified dataset

In [2]:
# define parameters to load data

# define the path to the data
data_path = '../unlabeled_data/'

# define the names of the files to load
file_names = ['T_FRRSA.csv', 'T_FRDEM.csv','T_FRUDSAB.csv',
              'T_FRDSM.csv','T_FRMDH.csv','T_FRPEX.csv',
              'T_FRPBC.csv','T_FRTFB.csv','T_FRDOS.csv', 
              'SAE.csv']

# define the names of the variables for the dataframes
variables = ['rsa', 'dem', 'uds', 'dsm', 'mdh', 'pex', 
             'pbc', 'tfb', 'dos', 'sae']

# create a loop to iterate through the files and load them into the notebook
for file_name, variable in zip(file_names, variables):
        globals()[variable] = pd.read_csv(data_path + file_name)
        print(f"{variable} shape: {globals()[variable].shape}") # print the shape of the dataframes

rsa shape: (27029, 12)
dem shape: (1920, 43)
uds shape: (24930, 66)
dsm shape: (1889, 26)
mdh shape: (1869, 89)
pex shape: (2779, 33)
pbc shape: (2691, 20)
tfb shape: (100518, 56)
dos shape: (160908, 19)
sae shape: (117, 66)


### Transform Attendence Table
- This table establishes the patient population and will serve as the primary table
- All subsequent tables will use a LEFT JOIN to add clinical data as columns to each patient ID
- This table requires feature engineering for `attendance` and `dropout` variables

In [3]:
# we will define the columns and labels that we need for each df and then transform the data

# set parameters for transformation
rsa_cols = ['patdeid','VISIT','RSA001']
rsa_labels = {'RSA001':'attendance'}

# the helper function will transform the data
rsa = helper.clean_df(rsa, rsa_cols, rsa_labels)

# fill nulls with 0, marking no attendance
rsa['attendance'] = rsa['attendance'].fillna(0)

# remove the followup visits from the main clinical data weeks 0 - 24
rsa = rsa[~rsa['VISIT'].isin([28, 32])]

# remove duplicate rows
rsa = rsa.drop_duplicates(subset=['patdeid', 'VISIT'], keep='first')

# observe shape and sample 5 observations
print(rsa.shape)
display(rsa)

(24217, 3)


,patdeid,VISIT,attendance
0,1,0,1.0
2,1,1,1.0
3,1,2,1.0
4,1,3,1.0
5,1,4,1.0
...,...,...,...
27022,1931,23,0.0
27023,1931,24,1.0
27026,1932,0,1.0
27027,1933,0,1.0


### Feature Engineering
Capture sessions attended per patient

In [4]:
# create df with count of attendance for each patient
attendence = rsa.groupby('patdeid')['attendance'].size().to_frame('attendance').reset_index()

attendence

,patdeid,attendance
0,1,25
1,2,25
2,3,25
3,4,25
4,5,1
...,...,...
1915,1930,1
1916,1931,25
1917,1932,1
1918,1933,1


In [5]:
# set parameters to flatten the df
start = 0 # include data starting from week 0
end = 24 # finish at week 24
step = 1 # include data for every week

# call function to flatten dataframe
rsa_flat = helper.flatten_dataframe(rsa, start, end, step)

# fill nulls with 0 for no attendance
rsa_flat = rsa_flat.fillna(0)

# visually inspect the data
rsa_flat

,patdeid,attendance_0,attendance_1,attendance_2,attendance_3,attendance_4,attendance_5,attendance_6,attendance_7,attendance_8,...,attendance_15,attendance_16,attendance_17,attendance_18,attendance_19,attendance_20,attendance_21,attendance_22,attendance_23,attendance_24
0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
4,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,1930,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1916,1931,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
1917,1932,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1918,1933,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Feature Engineering
We will create the feature for treatment dropout, an important metric.<br>
Patients who do not show attendence for the final 4 weeks of treatment<br>
they will be considered to have dropped out of treatment

In [6]:
# create feature for treatment dropout
# treatment dropout is defined when patient misses attendance for final 4 weeks of treatment
rsa_flat['dropout'] = (
                        rsa_flat 
                       .iloc[:,22:25] # look at the final 4 columns
                       .sum(axis=1) # sum the values 
                       .apply(lambda x: 1 if x == 0 else 0) # if the sum is 0, then the patient dropped out
                       )

print('The dropout ratio is:')
display(rsa_flat.dropout.value_counts(normalize=True))

The dropout ratio is:


dropout
1    0.620313
0    0.379688
Name: proportion, dtype: float64

### Transform Demographics Table

In [7]:
# set parameters for transformation
dem_cols = ['patdeid','DEM002','DEM003A','DEM003B1','DEM003B2','DEM004A','DEM004B',     
            'DEM004C','DEM004E','DEM004F']
dem_labels = {'DEM002':'dem_gender','DEM003A':'dem_spanish_origin','DEM003B1':'dem_mexican',        
              'DEM003B2':'dem_puerto_rican','DEM004A':'dem_amer_indian','DEM004B':'dem_asian','DEM004C':'dem_black','DEM004E':'dem_white','DEM004F':'dem_other'}

# the helper function will clean and transform the data
dem = helper.clean_df(dem, dem_cols, dem_labels)

# fill missing values with 0
dem.fillna(0, inplace=True)

# we will need to change the values to binary
# for all columns afer 2, if values are > 0, change to 1 else 0
for col in dem.columns[2:]:
    dem[col] = dem[col].apply(lambda x: 1 if x > 0 else 0)

print('dem dataframe with shape of', dem.shape, 'has been cleaned ')
display(dem.sample(5))

dem dataframe with shape of (1920, 10) has been cleaned 


,patdeid,dem_gender,dem_spanish_origin,dem_mexican,dem_puerto_rican,dem_amer_indian,dem_asian,dem_black,dem_white,dem_other
1588,1600,1.0,1,0,0,0,0,0,1,0
1622,1634,1.0,1,0,0,0,0,0,1,0
1730,1742,2.0,1,0,0,0,0,0,1,0
421,426,1.0,1,0,0,0,0,0,1,0
1277,1287,2.0,1,0,0,0,0,0,1,0


### Transform Urine Drug Screen Table
This table contains the data for most of the outcome metrics<br>
Stay tuned for feature engineering section towards the end of this table transformation<br>

In [8]:
# set parameters for transformation
uds_cols = ['patdeid','VISIT', 'UDS005', 'UDS006', 'UDS007', 'UDS008', 'UDS009', 'UDS010', 'UDS011', 'UDS012', 
            'UDS013']
uds_labels = {'UDS005':'test_Amphetamines', 'UDS006':'test_Benzodiazepines','UDS007':'test_Methadone', 
              'UDS008':'test_Oxycodone', 'UDS009':'test_Cocaine', 'UDS010':'test_Methamphetamine', 'UDS011':'test_Opiate300', 'UDS012':'test_Cannabinoids', 'UDS013':'test_Propoxyphene'}

# the helper function will clean and transform the data
uds = helper.clean_df(uds, uds_cols, uds_labels)

print('Dataframe uds with shape of', uds.shape, 'has been cleaned')
display(uds)


Dataframe uds with shape of (24930, 11) has been cleaned


,patdeid,VISIT,test_Propoxyphene,test_Amphetamines,test_Cannabinoids,test_Benzodiazepines,test_Methadone,test_Oxycodone,test_Cocaine,test_Methamphetamine,test_Opiate300
0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
24925,1931,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24926,1931,32,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
24927,1932,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
24928,1933,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [9]:
# dataframe is ready to be flattened

# set params for flattening
start = 0 # include data starting from week 0
end = 24 # finish at week 24
step = 1 # include data for every week

# call function to flatten dataframe
uds_flat = helper.flatten_dataframe(uds, start, end, step)

# fill missing values with 1, which counts as missed test, which is the same as a negative test
uds_flat.fillna(1, inplace=True)

# visually inspect the data
print('The clinical data was added in the form of',uds_flat.shape[1],'features')
print('Which includes tests for 8 different drug classes over 24 weeks')
display(uds_flat)

The clinical data was added in the form of 226 features
Which includes tests for 8 different drug classes over 24 weeks


,patdeid,test_Propoxyphene_0,test_Amphetamines_0,test_Cannabinoids_0,test_Benzodiazepines_0,test_Methadone_0,test_Oxycodone_0,test_Cocaine_0,test_Methamphetamine_0,test_Opiate300_0,...,test_Opiate300_23,test_Propoxyphene_24,test_Amphetamines_24,test_Cannabinoids_24,test_Benzodiazepines_24,test_Methadone_24,test_Oxycodone_24,test_Cocaine_24,test_Methamphetamine_24,test_Opiate300_24
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1930,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1913,1931,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1914,1932,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1915,1933,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Feature Engineering
The following metrics will be created to assess treatment success:<br>
- `responder` - a binary feature indicating if the patient responds to treatment, by testing negative for opiates for the final 4 weeks of treatment
- `tnt` - total negative tests, a measure of clinical benefit, count of negative tests over 24 weeks
- `cnt` - concsecutive negative tests, a measure of clinical benefit, count of consecutive negative tests over 24 weeks

In [10]:
# call the helper function to create the UDS features
uds_features = helper.uds_features(uds_flat)

# isolate the features df for merge with the clinical data
uds_features = uds_features[['patdeid','TNT','CNT','responder']]

print('The UDS features have been created with shape of', uds_features.shape)
display(uds_features.sample(10))

The UDS features have been created with shape of (1917, 4)


,patdeid,TNT,CNT,responder
1110,1121,18,10,1
1523,1538,1,1,0
1855,1872,0,0,0
329,333,0,0,0
1901,1918,25,25,1
1916,1934,0,0,0
312,316,0,0,0
702,709,0,0,0
54,55,5,2,0
1744,1759,13,6,0


### Transform DSM-IV Diagnosis Table

In [11]:
# set params for transformation
dsm_cols = ['patdeid','DSMOPI','DSMAL','DSMAM','DSMCA','DSMCO','DSMSE']
dsm_labels = {'DSMOPI':'dsm_opiates','DSMAL':'dsm_alcohol','DSMAM':'dsm_amphetamine',
              'DSMCA':'dsm_cannabis','DSMCO':'dsm_cocaine','DSMSE':'dsm_sedative'}

# call the helper function to clean the data
dsm = helper.clean_df(dsm, dsm_cols, dsm_labels)

# change values to binary
for col in dsm.columns[1:]:
    dsm[col] = dsm[col].apply(lambda x: 1 if x > 0 else 0)

# fill nulls with 0, where patient does not confirm diagnosis
dsm.fillna(0, inplace=True)

print('Dataframe dsm with shape of', dsm.shape, 'has been cleaned')
display(dsm[:5])

Dataframe dsm with shape of (1889, 7) has been cleaned


,patdeid,dsm_cannabis,dsm_cocaine,dsm_sedative,dsm_opiates,dsm_alcohol,dsm_amphetamine
0,1,1,1,1,1,1,1
1,2,1,1,1,1,1,1
2,3,1,1,1,1,1,1
3,4,1,1,1,1,1,1
4,5,0,0,0,0,0,0


### Transform Medical and Psychiatric History Table
We will track 18 different medical conditions

In [12]:
# set parameters for transformation
mdh_cols = ['patdeid','MDH001','MDH002','MDH003','MDH004','MDH005','MDH006','MDH007','MDH008','MDH009',
            'MDH010','MDH011A','MDH011B','MDH012','MDH013','MDH014','MDH015','MDH016','MDH017']
mdh_labels = {'MDH001':'mds_head_injury','MDH002':'mds_allergies','MDH003':'mds_liver_problems',
                'MDH004':'mds_kidney_problems','MDH005':'mds_gi_problems','MDH006':'mds_thyroid_problems',
                'MDH007':'mds_heart_condition','MDH008':'mds_asthma','MDH009':'mds_hypertension',
                'MDH010':'mds_skin_disease','MDH011A':'mds_opi_withdrawal','MDH011B':'mds_alc_withdrawal',
                'MDH012':'mds_schizophrenia','MDH013':'mds_major_depressive_disorder',
                'MDH014':'mds_bipolar_disorder','MDH015':'mds_anxiety_disorder','MDH016':'mds_sig_neurological_damage','MDH017':'mds_epilepsy'}

# call the helper function to clean the data
mdh = helper.clean_df(mdh, mdh_cols, mdh_labels)

# imputation strategy, fill missing values with 0, indicates no diagnosis
mdh.fillna(0, inplace=True)

# visually inspect the data
print('Dataframe mdh with shape of', mdh.shape, 'has been cleaned')
display(mdh[:5])

Dataframe mdh with shape of (1869, 19) has been cleaned


,patdeid,mds_liver_problems,mds_kidney_problems,mds_alc_withdrawal,mds_schizophrenia,mds_major_depressive_disorder,mds_bipolar_disorder,mds_anxiety_disorder,mds_sig_neurological_damage,mds_allergies,mds_gi_problems,mds_thyroid_problems,mds_heart_condition,mds_asthma,mds_hypertension,mds_skin_disease,mds_head_injury,mds_opi_withdrawal,mds_epilepsy
0,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Transform the PEX (Physical Exam) Table

In [13]:
# set params to clean cols
pex_cols = ['patdeid','PEX001A','PEX002A','PEX003A','PEX004A','PEX005A','PEX006A','PEX007A',
            'PEX008A','PEX009A','PEX010A','PEX011A','PEX012A']
pex_labels = {'PEX001A':'pex_gen_appearance','PEX002A':'pex_head_neck','PEX003A':'pex_ears_nose_throat',
              'PEX004A':'pex_cardio','PEX005A':'pex_lymph_nodes','PEX006A':'pex_respiratory',
              'PEX007A':'pex_musculoskeletal','PEX008A':'pex_gi_system','PEX009A':'pex_extremeties',
              'PEX010A':'pex_neurological','PEX011A':'pex_skin','PEX012A':'pex_other'}
              
# call the helper function to clean the data
pex = helper.clean_df(pex, pex_cols, pex_labels)

# remove duplicate rows
pex = pex.drop_duplicates(subset=['patdeid'], keep='first')

# imputation strategy: 9 indicates no diagnosis
pex.fillna(9, inplace=True)

# visually inspect the data
print('Dataframe pex with shape of', pex.shape, 'has been cleaned')
display(pex[:5])

Dataframe pex with shape of (1869, 13) has been cleaned


,patdeid,pex_lymph_nodes,pex_other,pex_respiratory,pex_musculoskeletal,pex_gi_system,pex_extremeties,pex_neurological,pex_gen_appearance,pex_ears_nose_throat,pex_head_neck,pex_cardio,pex_skin
0,1,1.0,9.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,2,1.0,9.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0
4,3,1.0,9.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
6,4,1.0,9.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,5,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0


### Transform the Pregnancy and Birth Control Table

In [14]:
# define parameters for cleaning
pbc_cols = ['patdeid','VISIT','PBC003']
pbc_labels = {'PBC003':'pbc_test_result'} 

# call the helper function to clean the data
pbc = helper.clean_df(pbc, pbc_cols, pbc_labels)

# remove followup visits from the main clinical after week 24
pbc = pbc[~pbc['VISIT'].isin([28, 32])]

pbc = pbc.fillna(0)

pbc

,patdeid,VISIT,pbc_test_result
0,4,0,1.0
1,4,4,1.0
2,4,8,1.0
3,4,12,1.0
4,4,16,1.0
...,...,...,...
2684,1921,24,1.0
2687,1923,0,1.0
2688,1923,4,1.0
2689,1923,8,1.0


In [15]:
# flatten the pbc data

# set parameters for flattening
start = 0 # include data starting from week 0
end = 24 # finish at week 24
step = 4 # include data for every week

pbc_flat = helper.flatten_dataframe(pbc, start, end, step)

pbc_flat


,patdeid,pbc_test_result_0,pbc_test_result_4,pbc_test_result_8,pbc_test_result_12,pbc_test_result_16,pbc_test_result_20,pbc_test_result_24
0,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,6,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,8,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,19,1.0,1.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
426,1906,1.0,1.0,1.0,1.0,1.0,0.0,1.0
427,1913,0.0,0.0,NaN,NaN,NaN,NaN,1.0
428,1918,1.0,1.0,2.0,1.0,1.0,1.0,1.0
429,1921,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Transform the TFB (Timeline Follow Back Survey) Table
- This table has an issue with multiple rows per patient
- After the table is cleaned, we will aggregate each row to a single row per patient
- After the aggregation, the table will be flattened, to encode the survey, drug class and week collected, in each column
- Surveys are collected once a month and reflect the previous 30 days of drug use

In [16]:
# define parameters for cleaning
tfb_cols = ['patdeid','VISIT','TFB001A','TFB002A','TFB003A','TFB004A','TFB005A','TFB006A','TFB007A',
            'TFB008A','TFB009A','TFB010A']
tfb_labels = {'TFB001A':'survey_alcohol','TFB002A':'survey_cannabis','TFB003A':'survey_cocaine',    
              'TFB010A':'survey_oxycodone','TFB009A':'survey_methadone','TFB004A':'survey_amphetamine','TFB005A':'survey_methamphetamine','TFB006A':'survey_opiates','TFB007A':'survey_benzodiazepines','TFB008A':'survey_propoxyphene'}

# call the helper function to clean the data
tfb = helper.clean_df(tfb, tfb_cols, tfb_labels)

# visually inspect the data
print('Shape of cleaned tfb dataframe is', tfb.shape)
display(tfb[:5])

Shape of cleaned tfb dataframe is (100518, 12)


,patdeid,VISIT,survey_cannabis,survey_cocaine,survey_alcohol,survey_oxycodone,survey_methadone,survey_amphetamine,survey_methamphetamine,survey_opiates,survey_benzodiazepines,survey_propoxyphene
0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
# aggregate rows by patient and visit, sum all records of drug use

# create index
index = ['patdeid','VISIT']

# create aggregation dictionary, omit the first two columns, they do not require aggregation
agg_dict = {col:'sum' for col in tfb.columns[2:]}

# aggregate the data, we will apply sum to all instances of reported us to give the total use for the period
tfb_agg = tfb.groupby(index).agg(agg_dict).reset_index()

# visually inspect the data
print('Aggregated tfb dataframe contains', tfb_agg.shape[0],'rows, coming from', tfb.shape[0],'rows')
display(tfb_agg[:5])

Aggregated tfb dataframe contains 6008 rows, coming from 100518 rows


,patdeid,VISIT,survey_cannabis,survey_cocaine,survey_alcohol,survey_oxycodone,survey_methadone,survey_amphetamine,survey_methamphetamine,survey_opiates,survey_benzodiazepines,survey_propoxyphene
0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0
1,1,4,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,1,8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0
4,2,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [18]:
# flatten the dataframe

# set parameters to flatten survey data
start = 0 # include data starting from week 0
end = 24 # finish at week 24
step = 4 # include data for every 4 weeks

# call function to flatten dataframe
tfb_flat = helper.flatten_dataframe(tfb_agg, start, end, step)

# imputation strategy: fill missing values with 0, indicates no drug use
tfb_flat.fillna(0, inplace=True)

# visualize the data
print('Flattended dataframe contains', tfb_flat.shape[1]-1,'features')
display(tfb_flat)

Flattended dataframe contains 70 features


,patdeid,survey_cannabis_0,survey_cocaine_0,survey_alcohol_0,survey_oxycodone_0,survey_methadone_0,survey_amphetamine_0,survey_methamphetamine_0,survey_opiates_0,survey_benzodiazepines_0,...,survey_cannabis_24,survey_cocaine_24,survey_alcohol_24,survey_oxycodone_24,survey_methadone_24,survey_amphetamine_24,survey_methamphetamine_24,survey_opiates_24,survey_benzodiazepines_24,survey_propoxyphene_24
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
2,3,0.0,23.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,...,0.0,4.0,4.0,0.0,0.0,0.0,0.0,28.0,1.0,0.0
3,4,1.0,2.0,0.0,1.0,0.0,0.0,0.0,30.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,6,0.0,0.0,0.0,25.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,1925,2.0,1.0,0.0,0.0,1.0,0.0,0.0,29.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1662,1927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1663,1929,10.0,1.0,4.0,5.0,0.0,0.0,0.0,16.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1664,1930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Transform the Medication Dose Table
- This table has an issue with multiple rows per patient
- Each dose of medication is recorded as a row
- This means that if a patient received 7 doses of medication, there will be 7 rows for that patient
- This needs to be consolidated into a single row per patient
- For total_dose with null values, we will treat that as a no show or 0 dose

In [19]:
# set parameters for cleaning the dataframe
dos_cols = ['patdeid','VISIT','DOS002','DOS005']   
dos_labels = {'DOS002':'medication','DOS005':'total_dose'}

# call the helper function to clean the data
dos = helper.clean_df(dos, dos_cols, dos_labels)

# Imputation strategy: backfill and forwardfill missing values from medication and total dose
dos['medication'] = dos['medication'].fillna(method='ffill').fillna(method='bfill')
dos['total_dose'] = dos['total_dose'].fillna(method='ffill').fillna(method='bfill')

# observe the data
print('The medication dataframe contains', dos.shape[0],'rows that must be aggregated')
display(dos)

The medication dataframe contains 160908 rows that must be aggregated


,patdeid,VISIT,medication,total_dose
0,1,0,2.0,8.0
1,1,1,2.0,16.0
2,1,1,2.0,24.0
3,1,1,2.0,24.0
4,1,1,2.0,32.0
...,...,...,...,...
160903,1931,24,2.0,8.0
160904,1931,24,2.0,8.0
160905,1931,24,2.0,8.0
160906,1931,24,2.0,8.0


In [20]:
# aggregate columns 

# create index
index = ['patdeid','VISIT','medication']
# create aggregation dictionary
agg_dict = {col:'sum' for col in dos.columns[3:]}

# aggregate the data, we will add daily dose to create weekly dose total, aggregating multiple columns per patient
dos_agg = dos.groupby(index).agg(agg_dict).reset_index()

# create df with patdeid and medication to merge later, this will help make analysis easier
medication = dos[['patdeid', 'medication']].drop_duplicates(subset=['patdeid'], keep='first').reset_index(drop=True)

# visualize the data
print('Total rows in the aggregated dataframe:', dos_agg.shape[0],'from', dos.shape[0],'rows')
dos_agg


Total rows in the aggregated dataframe: 23666 from 160908 rows


,patdeid,VISIT,medication,total_dose
0,1,0,2.0,8.0
1,1,1,2.0,160.0
2,1,2,2.0,320.0
3,1,3,2.0,192.0
4,1,4,2.0,384.0
...,...,...,...,...
23661,1931,20,2.0,60.0
23662,1931,21,2.0,48.0
23663,1931,22,2.0,56.0
23664,1931,23,2.0,0.0


### Feature Engineering
Create separate columns for bupe and methadone, this improves data quality

In [21]:
# feature engineering

# call helper function to create features from the medication data
dos_agg = helper.med_features(dos_agg)

# visually inspect the data
print('The aggregated dataframe contains', dos_agg.shape[1],'features')
display(dos_agg)

The aggregated dataframe contains 4 features


,patdeid,VISIT,meds_methadone,meds_buprenorphine
0,1,0,0.0,8.0
1,1,1,0.0,160.0
2,1,2,0.0,320.0
3,1,3,0.0,192.0
4,1,4,0.0,384.0
...,...,...,...,...
23661,1931,20,0.0,60.0
23662,1931,21,0.0,48.0
23663,1931,22,0.0,56.0
23664,1931,23,0.0,0.0


In [22]:
# flatten the dataframe

# set parameters to flatten the dataframe
start = 0 # include data starting from week 0
end = 24 # finish at week 24
step = 1 # include data for every week

# call function to flatten dataframe
dos_flat = helper.flatten_dataframe(dos_agg, start, end, step)

# imputation strategy: nulls come post merge, these were visits for patients who dropped out, fill with 0
dos_flat.fillna(0, inplace=True)

print('The flattened dataframe contains', dos_flat.shape[1]-1,'features')
display(dos_flat)

The flattened dataframe contains 50 features


,patdeid,meds_methadone_0,meds_buprenorphine_0,meds_methadone_1,meds_buprenorphine_1,meds_methadone_2,meds_buprenorphine_2,meds_methadone_3,meds_buprenorphine_3,meds_methadone_4,...,meds_methadone_20,meds_buprenorphine_20,meds_methadone_21,meds_buprenorphine_21,meds_methadone_22,meds_buprenorphine_22,meds_methadone_23,meds_buprenorphine_23,meds_methadone_24,meds_buprenorphine_24
0,1,0.0,8.0,0.0,160.0,0.0,320.0,0.0,192.0,0.0,...,0.0,210.0,0.0,180.0,0.0,246.0,0.0,128.0,0.0,166.0
1,2,0.0,8.0,0.0,48.0,0.0,48.0,0.0,60.0,0.0,...,0.0,40.0,0.0,72.0,0.0,60.0,0.0,72.0,0.0,68.0
2,3,30.0,0.0,170.0,0.0,310.0,0.0,420.0,0.0,360.0,...,600.0,0.0,670.0,0.0,630.0,0.0,510.0,0.0,540.0,0.0
3,4,0.0,16.0,0.0,152.0,0.0,192.0,0.0,160.0,0.0,...,0.0,256.0,0.0,32.0,0.0,160.0,0.0,128.0,0.0,32.0
4,6,0.0,16.0,0.0,16.0,0.0,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,1922,30.0,0.0,270.0,0.0,390.0,0.0,560.0,0.0,420.0,...,840.0,0.0,140.0,0.0,560.0,0.0,420.0,0.0,560.0,0.0
1478,1923,0.0,8.0,0.0,32.0,0.0,64.0,0.0,80.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1479,1925,30.0,0.0,690.0,0.0,1460.0,0.0,100.0,0.0,1400.0,...,700.0,0.0,100.0,0.0,1400.0,0.0,700.0,0.0,1097.0,0.0
1480,1929,110.0,0.0,270.0,0.0,250.0,0.0,300.0,0.0,360.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Transform the Serious Adverse Events Table

In [23]:
# set parameters to clean table
sae_cols = ['patdeid','SAE002','SAE006A','SAE006B','SAE006C','SAE006D']
sae_labels = {'SAE002':'sae_name','SAE006A':'sae_cat_death',
              'SAE006B':'sae_cat_life_threat','SAE006C':'sae_cat_hospitalization','SAE006D':'sae_cat_disability'}
              
# call the helper function to clean the data
sae = helper.clean_df(sae, sae_cols, sae_labels)

# fill missing values with 0, indicates no adverse event
# sae.fillna(0, inplace=True)

sae

,patdeid,sae_name,sae_cat_death,sae_cat_life_threat,sae_cat_hospitalization,sae_cat_disability
0,28.0,SPONTANEOUS ABORTION,0.0,0.0,0.0,0.0
1,76.0,CONCUSSION,0.0,0.0,0.0,1.0
2,88.0,RUPTURE OF PRE-EXISTING STOMACH ULCER,0.0,0.0,1.0,0.0
3,88.0,LUMBAR FRACTURE,0.0,0.0,1.0,0.0
4,88.0,STOMACH BLEEDING,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
112,1766.0,INFECTED ABDOMINAL LIPOMA,0.0,0.0,0.0,0.0
113,1791.0,RIGHT DELTOID ABSCESS (SKIN),0.0,0.0,1.0,0.0
114,1814.0,ABSCESS LEFT FOREARM,0.0,0.0,1.0,0.0
115,1814.0,LEFT UPPER ARM ABSCESS,0.0,0.0,1.0,0.0


### Now we will merge all the tables into a single dataset

In [24]:
# set parameters for merge

# Define the dataframes to merge
dfs = [rsa_flat, dos_flat, uds_flat, tfb_flat, 
       pbc_flat, uds_features, dem, dsm, mdh, 
       pex, sae, medication, attendence]

# Initialize merged_df with the first DataFrame in the list
merged_df = dfs[0]

# Merge the dfs above using left merge on 'patdeid'
for df in dfs[1:]:  # Start from the second item in the list
    merged_df = pd.merge(merged_df, df, on='patdeid', how='left')

# some rows were duplicated from one:many merge, they will be dropped
merged_df = merged_df.drop_duplicates(subset=['patdeid'], keep='first')

# Print the shape of the final dataframe
print('The final table includes', merged_df.shape[1], 'features for', merged_df.shape[0], 'patients in treatment')

The final table includes 434 features for 1920 patients in treatment


### Analyze Null Values

In [25]:
# show all rows from the function call
pd.set_option('display.max_rows', None)
merged_df.isnull().sum()

patdeid                             0
attendance_0                        0
attendance_1                        0
attendance_2                        0
attendance_3                        0
attendance_4                        0
attendance_5                        0
attendance_6                        0
attendance_7                        0
attendance_8                        0
attendance_9                        0
attendance_10                       0
attendance_11                       0
attendance_12                       0
attendance_13                       0
attendance_14                       0
attendance_15                       0
attendance_16                       0
attendance_17                       0
attendance_18                       0
attendance_19                       0
attendance_20                       0
attendance_21                       0
attendance_22                       0
attendance_23                       0
attendance_24                       0
dropout     

### Imputation Strategy

In [26]:
# for the medication column we will backfill and forward fill the nulls
# these patients dropped out, however, we would like to closely track their meds where possible
merged_df.medication = merged_df.medication.fillna(method='ffill').fillna(method='bfill')

# for columns that have 'meds' in the column name, forwardfill and backfill nulls
# these columns are the daily dose of medication
for col in merged_df.columns:
    if 'meds' in col:
        merged_df[col] = merged_df[col].fillna(method='ffill').fillna(method='bfill')

# for the sae and pbc columns, the nulls are just patients without data, can be set to 0
# for the pex columns, nulls come from patients who dropped from treatment
# can be filled with 0

# create list with prefix of columns to fill
cols = ['sae','pbc', 'pex', 'dsm', 'survey','mds',]
# if column has one of the elements in cols, fill with 0
for col in merged_df.columns:
    if any(x in col for x in cols):
        merged_df[col] = merged_df[col].fillna(0)

In [28]:
merged_df.loc[merged_df.test_Propoxyphene_0.isnull()]

,patdeid,attendance_0,attendance_1,attendance_2,attendance_3,attendance_4,attendance_5,attendance_6,attendance_7,attendance_8,...,pex_head_neck,pex_cardio,pex_skin,sae_name,sae_cat_death,sae_cat_life_threat,sae_cat_hospitalization,sae_cat_disability,medication,attendance
353,334,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,2.0,1
1093,1003,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,1
1096,1006,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,9.0,9.0,0,0.0,0.0,0.0,0.0,1.0,1


In [27]:
merged_df.isnull().sum()

patdeid                          0
attendance_0                     0
attendance_1                     0
attendance_2                     0
attendance_3                     0
attendance_4                     0
attendance_5                     0
attendance_6                     0
attendance_7                     0
attendance_8                     0
attendance_9                     0
attendance_10                    0
attendance_11                    0
attendance_12                    0
attendance_13                    0
attendance_14                    0
attendance_15                    0
attendance_16                    0
attendance_17                    0
attendance_18                    0
attendance_19                    0
attendance_20                    0
attendance_21                    0
attendance_22                    0
attendance_23                    0
attendance_24                    0
dropout                          0
meds_methadone_0                 0
meds_buprenorphine_0

In [ ]:
# save to data folder in csv
merged_df.to_csv('../data/merged_data.csv', index=False)